In [6]:
from src.constants.model_constants import window_size
from src.constants.data_constants import current_data_path
from src.models.utils import helper



In [7]:
df = helper.load_data(current_data_path)

values = df.values
n_train_hours = 7 * 8

train = values[:n_train_hours, :]
test = values[n_train_hours:, :]


train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]


train_X = train_X.reshape((train_X.shape[0], window_size, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], window_size, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

ValueError: cannot reshape array of size 336 into shape (84,12,4)